This notebook gathers all the results from previous notebooks (ones predicted from a dataset with missing values) and filled in the missing predictions by looking for the most similar items (same categories,etc.)

In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#save and load function
import pickle
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
  

In [3]:
test_goal = pd.read_csv("sample_submission.csv")

In [4]:
test_goal.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [5]:
test = pd.read_csv("test.csv")
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [7]:
test_target = test.set_index(["shop_id","item_id"])

In [8]:
test_target.head()

ID
shop_id item_id    
5       5037      0
        5320      1
        5233      2
        5232      3
        5268      4

In [8]:
#group by item id from the prediction

In [9]:
#load the test
test_pred_df = load_obj("data/test_pred_df_mlp")

In [10]:
test_pred_df.shop_id = test_pred_df.shop_id.astype(int)

In [11]:
test_pred_df.item_id = test_pred_df.item_id.astype(int)

In [12]:
test_pred = test_pred_df.copy()

In [13]:
test_pred = test_pred.set_index(["shop_id","item_id"])

In [14]:
test_pred.head()

item_category_id  top_categories_id  mlp_pred_block
shop_id item_id                                                     
0       30                   40.0               13.0        0.159649
        31                   37.0               13.0        0.158120
        32                   40.0               13.0        0.160011
        33                   37.0               13.0        0.158203
        35                   40.0               13.0        0.160009

In [15]:
test_merge = test_target.join(test_pred,how = "left")
max(test_merge["mlp_pred_block"])

0.19578083507063343

In [16]:
#test_pred = test_pred.reset_index()
test_pred.head()

item_category_id  top_categories_id  mlp_pred_block
shop_id item_id                                                     
0       30                   40.0               13.0        0.159649
        31                   37.0               13.0        0.158120
        32                   40.0               13.0        0.160011
        33                   37.0               13.0        0.158203
        35                   40.0               13.0        0.160009

In [17]:
test_merge.head()

ID  item_category_id  top_categories_id  mlp_pred_block
shop_id item_id                                                         
5       5037      0              19.0                6.0        0.186415
        5320      1               NaN                NaN             NaN
        5233      2              19.0                6.0        0.186415
        5232      3              23.0                6.0        0.186415
        5268      4               NaN                NaN             NaN

##### To check why there are so many NAN

In [18]:
#a lot of NaNs
#check if every item is in test_pred
index_target = test_target.index
index_pred = test_pred.index

In [19]:
len(index_target.isin(index_pred))

214200

In [20]:
from collections import Counter

In [21]:
Counter(list(index_target.isin(index_pred)))

Counter({True: 111404, False: 102796})

In [22]:
#check the original data
train_merged = load_obj("data/train_merged")

In [23]:
train_merged_shop_item = train_merged[["shop_id","item_id","item_cnt_day"]]

In [24]:
train_merged_shop_item = train_merged_shop_item.set_index(["shop_id","item_id"])

In [25]:
original_index = train_merged_shop_item.index

In [26]:
Counter(list(index_target.isin(original_index)))

Counter({True: 111404, False: 102796})

In [27]:
train_raw = pd.read_csv("sales_train_v2.csv")

In [28]:
train = train_raw.set_index(["shop_id","item_id"])

In [29]:
first_index = train.index

In [30]:
Counter(list(index_target.isin(first_index)))

Counter({True: 111404, False: 102796})

In [31]:
train_raw.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [32]:
train_raw["item_cnt_day"].describe()

count    2.935849e+06
mean     1.242641e+00
std      2.618834e+00
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.169000e+03
Name: item_cnt_day, dtype: float64

In [33]:
save_obj(test_merge,"data/final_result_mlp")

In [34]:
test_merge = load_obj("data/final_result_mlp")

So these items were not in the train set in the first place.

The missing values will be predicted by the mean of same item sold in other shops.

If it was never sold in the history of the data, the sales amount will be calculated based on its closest category in this shop.

If this doesn't work then search for the top category, if still fails then fill in the value with with the mean sale prediction amount of all available data.

In [35]:
test_merge.head()

ID  item_category_id  top_categories_id  mlp_pred_block
shop_id item_id                                                         
5       5037      0              19.0                6.0        0.186415
        5320      1               NaN                NaN             NaN
        5233      2              19.0                6.0        0.186415
        5232      3              23.0                6.0        0.186415
        5268      4               NaN                NaN             NaN

In [36]:
test_merge = test_merge.reset_index()

In [37]:
test_merge_NA = test_merge.loc[test_merge["mlp_pred_block"].isnull()]

In [38]:
test_merge_NA.head()

,shop_id,item_id,ID,item_category_id,top_categories_id,mlp_pred_block
1,5,5320,1,NaN,NaN,NaN
4,5,5268,4,NaN,NaN,NaN
9,5,5003,9,NaN,NaN,NaN
16,5,4874,16,NaN,NaN,NaN
21,5,5002,21,NaN,NaN,NaN


In [39]:
test_merge_not_NA =  test_merge.loc[~test_merge["mlp_pred_block"].isnull()]

In [40]:
test_merge_not_NA.head()

,shop_id,item_id,ID,item_category_id,top_categories_id,mlp_pred_block
0,5,5037,0,19.0,6.0,0.186415
2,5,5233,2,19.0,6.0,0.186415
3,5,5232,3,23.0,6.0,0.186415
5,5,5039,5,23.0,6.0,0.186415
6,5,5041,6,20.0,6.0,0.186415


In [41]:
fill_na_value = np.mean(test_merge_not_NA["mlp_pred_block"])

In [42]:
fill_na_value 

0.18618592851357244

In [43]:
items_categories_merged = load_obj("data/items_categories_merged")

In [44]:
items_categories_merged  = items_categories_merged.drop(columns = ["item_category_name","item_category_name_translated","Top Categories"])

In [45]:
items = pd.read_csv("items.csv")

In [46]:
items = items.drop(columns = ["item_name"])

In [47]:
items_joined = items.merge(items_categories_merged,left_on = "item_category_id", right_on = "item_category_id")

In [48]:
items_joined.head()

,item_id,item_category_id,top_categories_id
0,0,40,13
1,2,40,13
2,3,40,13
3,4,40,13
4,5,40,13


In [49]:
test_merge_NA = test_merge_NA.merge(items_joined,left_on = "item_id",right_on = "item_id")

In [50]:
test_merge_NA = test_merge_NA.drop(columns = ["item_category_id_x","top_categories_id_x","mlp_pred_block"])

In [51]:
test_merge_NA.head()

,shop_id,item_id,ID,item_category_id_y,top_categories_id_y
0,5,5320,1,55,15
1,4,5320,5101,55,15
2,6,5320,10201,55,15
3,3,5320,15301,55,15
4,2,5320,20401,55,15


In [52]:
test_merge_NA.columns = [
"shop_id",
"item_id",
"ID",
"item_category_id",
"top_categories_id"]

In [53]:
test_merge_NA["item_cnt_month"] = -999

In [54]:
test_merge_NA.head()

,shop_id,item_id,ID,item_category_id,top_categories_id,item_cnt_month
0,5,5320,1,55,15,-999
1,4,5320,5101,55,15,-999
2,6,5320,10201,55,15,-999
3,3,5320,15301,55,15,-999
4,2,5320,20401,55,15,-999


In [55]:
#save the NA rows
save_obj(test_merge_NA,"data/test_merge_NA")

In [56]:
import math

In [57]:
for index, row in test_merge_NA.iterrows():
    shop_id = row["shop_id"]
    item_id = row["item_id"]
    cat1 = row["item_category_id"]
    cat2 = row["top_categories_id"]
    temp = 0
    #fill in with the mean of the items available in other shops, if it's available
    #emp = 0
    
    temp = np.mean(test_merge_not_NA.loc[test_merge_not_NA["item_id"] == item_id,"mlp_pred_block"])
    
    if not (math.isnan(temp)):
        test_merge_NA.loc[index,"item_cnt_month"] = temp
    else:
        temp_df = test_merge_not_NA.loc[test_merge_not_NA["shop_id"] == shop_id]
        temp = np.mean(temp_df.loc[temp_df["item_category_id"] == cat1,"mlp_pred_block"])
        if not (math.isnan(temp)):
            test_merge_NA.loc[index,"item_cnt_month"] = temp
        else:
            temp_df = test_merge_not_NA.loc[test_merge_not_NA["shop_id"] == shop_id]
            temp = np.mean(temp_df.loc[temp_df["top_categories_id"] == cat2,"mlp_pred_block"])
            if not (math.isnan(temp)):
                test_merge_NA.loc[index,"item_cnt_month"] = temp
            else:
                #mean of all the categories in all shops
                temp = np.mean(test_merge_not_NA.loc[test_merge_not_NA["item_category_id"] == cat1,"mlp_pred_block"])
                if not (math.isnan(temp)):
                    test_merge_NA.loc[index,"item_cnt_month"] = temp
                else:
                    temp = np.mean(test_merge_not_NA.loc[test_merge_not_NA["top_categories_id"] == cat2,"mlp_pred_block"])
                    if not (math.isnan(temp)):
                        test_merge_NA.loc[index,"item_cnt_month"] = temp 
                    else:
                        #last resort
                        test_merge_NA.loc[index,"item_cnt_month"] = fill_na_value
    

In [58]:
test_merge_NA_test = test_merge_NA.iloc[0] 

In [59]:
shop_id = test_merge_NA_test["shop_id"]
item_id = test_merge_NA_test["item_id"]
cat1 = test_merge_NA_test["item_category_id"]
cat2 = test_merge_NA_test["top_categories_id"]

In [60]:
temp = np.mean(test_merge_not_NA.loc[test_merge_not_NA["item_id"] == item_id,"mlp_pred_block"])

In [61]:
not math.isnan(temp)

False

In [62]:
test_merge_NA.head()

,shop_id,item_id,ID,item_category_id,top_categories_id,item_cnt_month
0,5,5320,1,55,15,0.186415
1,4,5320,5101,55,15,0.186415
2,6,5320,10201,55,15,0.186415
3,3,5320,15301,55,15,0.186415
4,2,5320,20401,55,15,0.186415


In [63]:
test_merge_NA["item_cnt_month"].describe()

count    102796.000000
mean          0.185488
std           0.006077
min           0.119467
25%           0.186415
50%           0.186415
75%           0.186415
max           0.195781
Name: item_cnt_month, dtype: float64

In [64]:
cols = test_merge_NA.columns
test_merge_not_NA.columns = cols

In [65]:
#concat the results
result_MLP_postKNN = pd.concat([test_merge_NA,test_merge_not_NA])

In [66]:
result_MLP_postKNN  = result_MLP_postKNN.sort_values(by = "ID")

In [67]:
result_MLP_postKNN = result_MLP_postKNN[["ID","item_cnt_month"]]

In [68]:
result_MLP_postKNN["item_cnt_month"].head()

0     0.186415
0     0.186415
2     0.186415
3     0.186415
42    0.186415
Name: item_cnt_month, dtype: float64

In [69]:
result_MLP_postKNN["item_cnt_month"].isnull().unique()

array([False])

In [70]:
result_MLP_postKNN.to_csv("result_MLP_postKNN.csv",index = False)